In [1]:
import numpy as np
import pandas as pd
import import_ipynb
import misc_functions as msc_fct
import random
import time

importing Jupyter notebook from misc_functions.ipynb


In [2]:
def sim_orders(dt, Ndt, NSIM, kappa, lamb):
    '''
    Simulates the arrival of market buy orders.
    
    imputs:
    dt: time step
    Ndt: number of time steps
    NSIM: number of simmulations
    kappa: intensity of exponential distribution
    lamb: intensity of poission process
    
    returns:
    sim: 2D numpy array of depths reached by buy orders for
    each simulation and time period
    '''
    
    # pre processing various matricies
    sim = np.full((NSIM,Ndt), np.nan)
    rand = np.random.rand(NSIM, Ndt)
    
    # giving depth reached by buy orders
    exp = np.random.exponential(1/kappa,(NSIM,Ndt))
    
    # probability of buy order arriving
    P = 1 - np.exp(-lamb*dt)
    
    # boolian array 1 if order arrives
    arrive = np.less(rand, P)
    
    for k in range(0,NSIM):
        idx = arrive[k,:]
        sim[k,idx] = exp[k,idx]
    
    # Adding a collumn of nan for t=0
    sim = np.insert(sim, 0, np.nan, axis=1)
    
    return sim

In [3]:
def discretisation(sim, sections):
    '''
    Splits up the depths reached by buy orders depending on 
    the frequency of arrival
    
    imput:
    sim: 2D numpy array of depths reached by buy orders
    sections: how many desired sections to split the depth
    into
    
    output:
    bins: cut off of each section
    mid: midpoint of each section
    '''
    
    # turns 2d numpy array into 1 d array
    sim_1d = sim.flatten()
    sim_1d = pd.DataFrame(sim_1d)
    
    # splits in order to have even fequency in each bin
    sim_1d = pd.qcut(sim_1d.loc[:,0], q=(sections), labels=False, retbins = True)
    bins = sim_1d[1]
    
    mid = np.full((bins.size-1), np.nan)
    
    # producing mid value of bins
    for k in range(1, bins.size, 1):
        mid[k-1] = (bins[k-1] + bins[k])/2
    
    return bins, mid

In [4]:
def state_action(t, q, Ndt, Nq, mid):
    '''
    Produces a dictionary of all state action pairs
    
    imput:
    t: array of times
    q: range of quantities of inventory
    Ndt: number of time steps
    Nq: starting inventory
    mid: range of depths that limit orders can be posted
    
    output:
    state_actions_pairs: dictonary with state values as keys
    and actions as values
    '''
    
    states = []
    
    
    # loop to get all possible states
    for k in range(1,Ndt+1,1):
        for i in range(0,Nq+1,1):
            state = (t[k],q[i])    
            states.append(state)
            
    state_actions_pairs={}
    
    # loop to add actions to states
    for k in range(0,len(states),1):
        if states[k][1]==0:
            state_actions_pairs[states[k]] = [(0,False)] # (a,b) a - market orders, b - depths of limit orders
        else:
            actions = []
            for i in range(1,states[k][1]+2,1):
                if i-1 != states[k][1]:
                    for m in range(0,len(mid),1):
                        actions.append((i-1,mid[m]))
                else:
                    actions.append((i-1,False))
                    
            state_actions_pairs[states[k]] = actions
            
    return state_actions_pairs

In [5]:
def run_policy(policy, simulation, S0, Nq, Ndt, dt, xi, sigma):
    '''
    Runs a given policy
    
    imputs:
    policy: dictionary contaning the policy to be run
    simulation: the simulated buy orders reccieved
    S0: inital mid price
    Nq: inital number of inventory to be liquidated
    Ndt: the number of time steps
    dt: time step size
    xi: half spread
    sigma: variation of midprice
    
    output:
    X: terminal value of wealth process
    Q: inventory trajectory
    '''
    
    # ensuring when market buy order arives this is denonted by - inf
    simulation = msc_fct.nan_overwrite(simulation, -np.inf)
    
    # preconditioning
    Q = np.full(Ndt+1,np.nan)
    Q[0] = Nq
    S = S0   
    X = 0
    
    #running over all time steps
    for i in range(1,Ndt+1,1):
        #selecting given policy for each time step
        action = policy[(i*dt, Q[i-1])]
        #If limit order is placed
        if action[1] != False:
            #if filled
            if simulation[i] > action[1]:
                X = X + (S - xi) * action[0] + (S + action[1])
                Q[i] = Q[i-1] - action[0] - 1
            #if not filled    
            else:
                X = X + (S - xi) * action[0]
                Q[i] = Q[i-1] - action[0]
        #if no limit order placed        
        else:
            X = X + (S - xi) * action[0]
            Q[i] = Q[i-1] - action[0]
            S = S + sigma*np.random.normal(1)
    #if terminal inventory remains liquidate it        
    if Q[-1] != 0:
        X = X + (S - xi)*Q[-1]
        Q[-1] = 0
    return X, Q

In [6]:
def performance(X, Q, q_target, phi, Ndt, dt):
    '''
    Computes the performance  for a sample outupt
    
    imputs:
    X: terminal value of wealth process
    Q: trajectory of inventory
    q_target: target inventory trajectory
    phi: weight placed on deviations from inventory target
    Ndt: number of time steps
    dt: time step
    
    output:
    perf: performace value
    '''
    
    if q_target[0] != None:
        perf = X - phi*(np.cumsum(np.power(Q-q_target,2)*dt))[Ndt]
    else:
        perf = X
    
    return perf

In [11]:
def rand_policy(state_actions, Ndt, dt, Nq):
    '''
    Produces a random policy
    
    imputs:
    state_actions: all possible state action pairs
    Ndt: number of time steps
    dt: time step size
    Nq: number of units of inventory
    
    outputs:
    policy: a random policy
    '''
    policy = {}

    # getting random state action pairs for all other possible states and actions
    for i in range(1, Ndt+1, 1):
        for k in range(0, Nq + 1, 1):
            policy[(i*dt, k)] = random.choice(state_actions[(i*dt, k)])
            
    return policy

In [ ]:
def get_policies(policy, state_actions, Ndt, dt, Nq, confidence):
    ''' 
    Returns an array of possible policies in a format that can be used in
    the optimum policy
    
    imput:
    policy: the inital policy that will be used to produce other policies
    state_actions: all possible states and actions
    
    
    '''
    
    policy_values = np.array([policy, 0, 0, 0])
    policy_values.shape = (1,4)
    p = dict(policy)
    m = 0
    
    while m < confidence:
        p = dict(policy)
        p[(dt, Nq)] = random.choice(state_actions[(dt, Nq)])
        
        for i in range(2, Ndt + 1, 1):
        
            for k in range(0, Nq+1, 1):
                action = random.choice(state_actions[(i*dt, k)])
                p[(i*dt, k)] = action
    
        idx = policy_values[:,0] == p
        if sum(idx) == 0:
            policy_values = np.vstack((policy_values, np.array([p,0,0,0])))
        
        m = m + 1
    
    return policy_values

In [ ]:
def optimum_policy(policy_values, epsilon, state_actions, Ndt, dt, Nq, S0, sigma, xi, sim, q_target, phi, conf, comp):
    '''
    Produces an optimal policy
    
    imputs:
    policy_values: the policy values
    state_actions: state action pairs
    Ndt: number of time steps
    dt: time step size
    Nq: number of units of inventory
    S0: inital midprice price
    sigma: volatility of midprice
    xi: half bid ask spread i.e. cost of executing market orders
    sim: data points
    q_target: target liquidation schedule
    phi: parameter effecting how strongly agent penalised for deviating from target liquidation schedule
    conf: minimum number of times to run each policy
    comp: number of policies to compare optimal policy to
    
    outputs:
    pol: optimal policy
    value: performance criteria for optimal policy
    N: number of times optimal policy is run
    t1: time taken to find optimal policy
    res: policy, performance criteria and number of times run for each of the policies being compared to the optimal one
    '''
    t0 = time.clock()
    
    sim = msc_fct.nan_overwrite(sim, -np.inf)
    Q = np.full(Ndt+1, np.nan)
    Q[0] = Nq
    S = S0
    X = 0
    
    for m in range(0, len(policy_values[:,1]), 1):
        for k in range(0, conf, 1):
            simulation = sim[k,:]
            Q = np.full(Ndt + 1, np.nan)
            Q[0] = Nq
            S = S0
            X = 0
            policy = policy_values[m,0]
            X, Q = run_policy(policy, simulation, S0, Nq, Ndt, dt, xi, sigma)
            V = performance(X, Q, q_target, phi, Ndt, dt)
            idx = policy_values[:,0] == policy
            
            if k == 0:    
                policy_values[idx,1] = policy_values[idx,1]*(policy_values[idx,2]/(policy_values[idx,2] + 1)) + \
                (1/(policy_values[idx,2] + 1))*V
            
                policy_values[idx,2] = policy_values[idx,2] + 1 
            else:
                policy_values[idx,3] = policy_values[idx,3] + (V - policy_values[idx,1])*(V - policy_values[idx,1]*(policy_values[idx,2]/(policy_values[idx,2] + 1)) + \
                (1/(policy_values[idx,2] + 1))*V)
                
                policy_values[idx,1] = policy_values[idx,1]*(policy_values[idx,2]/(policy_values[idx,2] + 1)) + \
                (1/(policy_values[idx,2] + 1))*V
                
                policy_values[idx,2] = policy_values[idx,2] + 1 


    for k in range(conf + 1, len(sim[:,0]), 1):
        
        z = 0
    
        while policy_values[np.where(policy_values[:,2] == np.amin(policy_values[:,2])),2][0,0] < conf:
        
            simulation = sim[z,:]
            Q = np.full(Ndt + 1, np.nan)
            Q[0] = Nq
            S = S0
            X = 0
        
            policy = policy_values[np.where(policy_values[:,2] == np.amin(policy_values[:,2])),0][0,0]
            X, Q = run_policy(policy, simulation, S0, Nq, Ndt, dt, xi, sigma)
            V = performance(X, Q, q_target, phi, Ndt, dt)
            
            idx = policy_values[:,0] == policy
            
            if z == 0:    
                policy_values[idx,1] = policy_values[idx,1]*(policy_values[idx,2]/(policy_values[idx,2] + 1)) + \
                (1/(policy_values[idx,2] + 1))*V
            
                policy_values[idx,2] = policy_values[idx,2] + 1 
            else:
                policy_values[idx,3] = policy_values[idx,3] + (V - policy_values[idx,1])*(V - policy_values[idx,1]*(policy_values[idx,2]/(policy_values[idx,2] + 1)) + \
                (1/(policy_values[idx,2] + 1))*V)
                
                policy_values[idx,1] = policy_values[idx,1]*(policy_values[idx,2]/(policy_values[idx,2] + 1)) + \
                (1/(policy_values[idx,2] + 1))*V
                
                policy_values[idx,2] = policy_values[idx,2] + 1 
            
            z = z + 1
        
       
    
        simulation = sim[k,:]
        Q[0] = Nq
        S = S0
        X = 0
    
        policy = policy_values[np.where(policy_values[:,1] == np.amax(policy_values[:,1])),0][0,0]
        p = dict(policy)  
    
        for i in range(1, Ndt+1, 1):
            #selecting given action for each time step via epsilion greedy approach
            if np.random.rand(1) < epsilon:
                action = random.choice(state_actions[(i*dt, Q[i-1])])
            else:
                action = policy[(i*dt, Q[i-1])]
            
            p[(i*dt, Q[i-1])] = action
    
            #If limit order is placed
            if action[1] != False:
                #if filled
                if simulation[i] > action[1]:
                    X = X + (S - xi) * action[0] + (S + action[1])
                    Q[i] = Q[i-1] - action[0] - 1
                #if not filled    
                else:
                    X = X + (S - xi) * action[0]
                    Q[i] = Q[i-1] - action[0]
            #if no limit order placed        
            else:
                X = X + (S - xi) * action[0]
                Q[i] = Q[i-1] - action[0]
                S = S + sigma*np.random.normal(1)
        #if terminal inventory remains liquidate it        
        if Q[-1] != 0:
            X = X + (S - xi)*Q[-1]
            Q[-1] = 0
        
        V = performance(X, Q, q_target, phi, Ndt, dt)

        idx = policy_values[:,0] == p
        
        if sum(idx) > 0:
            policy_values[idx,3] = policy_values[idx,3] + (V - policy_values[idx,1])*(V - policy_values[idx,1]*(policy_values[idx,2]/(policy_values[idx,2] + 1)) + \
            (1/(policy_values[idx,2] + 1))*V)
                
            policy_values[idx,1] = policy_values[idx,1]*(policy_values[idx,2]/(policy_values[idx,2] + 1)) + \
            (1/(policy_values[idx,2] + 1))*V
                
            policy_values[idx,2] = policy_values[idx,2] + 1
            
        else:
            policy_values = np.vstack((policy_values, np.array([p,V,1,0])))
    
    
    pol = policy_values[np.where(policy_values[:,1] == np.amax(policy_values[:,1])),0][0,0]
    value = policy_values[np.where(policy_values[:,1] == np.amax(policy_values[:,1])),1][0,0]
    N = policy_values[np.where(policy_values[:,1] == np.amax(policy_values[:,1])),2][0,0]
        
    t1 = time.clock() - t0
    
    comparision = np.partition(policy_values[:,1], comp)[-comp:]
    
    res = np.full((comp,3),np.nan)
    for i in range(0,comp,1):
        res[i,0] = policy_values[np.where(policy_values[:,1] == comparision[i]),1][0,0]
        res[i,1] = policy_values[np.where(policy_values[:,1] == comparision[i]),2][0,0]
        res[i,2] = policy_values[np.where(policy_values[:,1] == comparision[i]),3][0,0]
    
    return pol, value, N, t1, res